<a href="https://colab.research.google.com/github/NguyenIsHere/Google-colab/blob/main/FoodModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

print("✅ TensorFlow version:", tf.__version__)
print("🚀 GPU:", "Có GPU" if tf.config.list_physical_devices('GPU') else "Không có GPU")


✅ TensorFlow version: 2.18.0
🚀 GPU: Có GPU


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install google_images_download bing-image-downloader


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 40.8 MB/s eta 0:00:00
  Created wheel for google_images_download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14537 sha256=527ef0c84067ac71df02df9cd3bded30b9251beedc4a0bead6b821ebdfcbfecb
  Stored in directory: /root/.cache/pip/wheels/dc/83/37/7303b15f3e8a5bfbd5c7ebbfe13f0c666ada6f8efecc6d77ec
Successfully built google_images_download


In [ ]:
from bing_image_downloader import downloader

food_list = ["Pho", "Banh Mi", "Bun Bo Hue", "Com Tam", "Goi Cuon"]

for food in food_list:
    downloader.download(food, limit=100, output_dir='dataset', adult_filter_off=True, force_replace=False)


[%] Downloading Images to /content/dataset/Pho


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://www.recipetineats.com/wp-content/uploads/2019/04/Beef-Pho_6.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://www.onceuponachef.com/images/2023/02/pho-1200x1478.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://images.squarespace-cdn.com/content/v1/52d3fafee4b03c7eaedee15f/11cd4cd9-9a5e-426f-a91a-f64bdf84812f/2023_06_09EOS+M506018.jpg
[Error]Invalid image, not saving https://images.squarespace-cdn.com/content/v1/52d3fafee4b03c7eaedee15f/11cd4cd9-9a5e-426f-a91a-f64bdf84812f/2023_06_09EOS+M506018.jpg

[!] Issue getting: https://images.squarespace-cdn.com/content/v1/52d3fafee4b03c7eaedee15f/11cd4cd9-9a5e-426f-a91a-f64bdf84812f/2023_06_09EOS+M506018.jpg
[!] Error:: Invalid image, not saving https://images.squarespace-cdn.com/content/v1/52d3fafee4b03c7eaedee15f/11cd4cd9-9a5e-426f-a91a-f64bdf84812f/2023_06_09EOS+M506018.

In [ ]:
import cv2
import os

dataset_path = "dataset/"
target_size = (224, 224)  # Hoặc (512, 512)

# Resize tất cả ảnh
for food in os.listdir(dataset_path):
    food_folder = os.path.join(dataset_path, food)
    for img_name in os.listdir(food_folder):
        img_path = os.path.join(food_folder, img_name)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, target_size)
        cv2.imwrite(img_path, img_resized)  # Lưu lại ảnh đã resize


In [ ]:
import os
import shutil
import random

dataset_path = "dataset"
output_path = "data_split"

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

for food in os.listdir(dataset_path):
    food_path = os.path.join(dataset_path, food)
    images = os.listdir(food_path)
    random.shuffle(images)

    train_split = int(len(images) * train_ratio)
    val_split = int(len(images) * (train_ratio + val_ratio))

    os.makedirs(f"{output_path}/train/{food}", exist_ok=True)
    os.makedirs(f"{output_path}/val/{food}", exist_ok=True)
    os.makedirs(f"{output_path}/test/{food}", exist_ok=True)

    for i, img in enumerate(images):
        src = os.path.join(food_path, img)
        if i < train_split:
            dst = os.path.join(f"{output_path}/train/{food}", img)
        elif i < val_split:
            dst = os.path.join(f"{output_path}/val/{food}", img)
        else:
            dst = os.path.join(f"{output_path}/test/{food}", img)

        shutil.copy(src, dst)

print("✅ Hoàn thành chia dataset!")


✅ Hoàn thành chia dataset!


In [ ]:
import tensorflow as tf

train_dir = "data_split/train"
val_dir = "data_split/val"
test_dir = "data_split/test"

img_size = (224, 224)
batch_size = 8

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size
)

print("✅ Dataset đã được load vào TensorFlow!")


Found 394 files belonging to 5 classes.
Found 50 files belonging to 5 classes.
Found 46 files belonging to 5 classes.
✅ Dataset đã được load vào TensorFlow!


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

print("✅ Hoàn thành tiền xử lý ảnh!")


✅ Hoàn thành tiền xử lý ảnh!


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Tránh overfitting
    layers.Dense(len(food_list), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Không train lại weights của MobileNetV2

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(food_list), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
epochs = 8  # Số epoch train

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

print("✅ Training hoàn tất!")


Epoch 1/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 214ms/step - accuracy: 0.4294 - loss: 1.5704 - val_accuracy: 0.7800 - val_loss: 0.6807
Epoch 2/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8223 - loss: 0.5235 - val_accuracy: 0.8000 - val_loss: 0.5448
Epoch 3/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8929 - loss: 0.2780 - val_accuracy: 0.8400 - val_loss: 0.4864
Epoch 4/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9199 - loss: 0.2077 - val_accuracy: 0.8400 - val_loss: 0.5170
Epoch 5/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9521 - loss: 0.1442 - val_accuracy: 0.8400 - val_loss: 0.5188
Epoch 6/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9491 - loss: 0.1402 - val_accuracy: 0.8400 - val_loss: 0.4563
Epoch 7/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9732 - loss: 0.0780 - val_accuracy: 0.8400 - val_loss: 0.4752
Epoch 8/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9701 - loss: 0.0876 - val_accuracy: 0.8400 - val_los

In [ ]:
model.save("food_model.keras")  # Định dạng Keras mới
print("✅ Mô hình đã được lưu!")


✅ Mô hình đã được lưu!


In [ ]:
import tensorflow as tf

# Load model từ file .h5 hoặc folder
model = tf.keras.models.load_model("food_model.keras")  # Nếu bạn lưu dạng .keras
# model = tf.keras.models.load_model("food_model")  # Nếu lưu dạng folder


In [ ]:
import cv2
import numpy as np

# Danh sách các nhãn (class) theo thứ tự lúc train
food_classes = ['Banh Mi', 'Bun Bo Hue', 'Com Tam', 'Goi Cuon', 'Pho']

# Load ảnh test
img_path = "data_split/test/Bun Bo Hue/Image_65.jpg"  # Đổi thành ảnh bạn muốn test
img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))  # Resize về đúng kích thước
img = img / 255.0  # Chuẩn hóa (nếu đã dùng Rescaling khi train)
img = np.expand_dims(img, axis=0)  # Thêm batch dimension

# Dự đoán
predictions = model.predict(img)
predicted_class = np.argmax(predictions)  # Lấy class có xác suất cao nhất

print(f"🍽 Món ăn dự đoán: {food_classes[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
🍽 Món ăn dự đoán: Bun Bo Hue


In [ ]:
loss, acc = model.evaluate(test_ds)
print(f"🎯 Độ chính xác trên tập test: {acc:.2%}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 622ms/step - accuracy: 0.8871 - loss: 0.4478
🎯 Độ chính xác trên tập test: 89.13%


In [ ]:
import os

dataset_path = "dataset"
for food in os.listdir(dataset_path):
    food_path = os.path.join(dataset_path, food)
    print(f"{food}: {len(os.listdir(food_path))} ảnh")


Goi Cuon: 100 ảnh
Com Tam: 100 ảnh
Pho: 100 ảnh
Bun Bo Hue: 100 ảnh
Banh Mi: 100 ảnh


In [ ]:
import tensorflow as tf

train_dir = "data_split/train"
class_names = sorted(os.listdir(train_dir))  # Lấy danh sách thư mục theo đúng thứ tự TensorFlow
print("Danh sách class theo thứ tự TensorFlow:", class_names)


Danh sách class theo thứ tự TensorFlow: ['Banh Mi', 'Bun Bo Hue', 'Com Tam', 'Goi Cuon', 'Pho']
